Using a DTS Resolver
==================

*This example is powered up thanks to the API available at Alpheios.net*


In [1]:
# Just importing the parent module, do not change this if you execute the code
# from the git repository
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


## Create the resolver

With the following line we create the resolver :

In [2]:
from MyCapytain.resolvers.dts.api_v1 import HttpDtsResolver

resolver = HttpDtsResolver("http://texts.alpheios.net/api/dts")

## Require metadata : let's visit the catalog

The following code is gonna find each text that is readable by Alpheios

In [3]:
# We get the root collection
root = resolver.getMetadata()
# Then we retrieve dynamically all the readableDescendants : it browse automatically the API until
# it does not have seen any missing texts: be careful with this one on huge repositories
readable_collections = root.readableDescendants
print("We found %s collections that can be parsed" % len(readable_collections))

We found 34 collections that can be parsed


### Printing the full tree

In [9]:
# Note that we could also see and make a tree of the catalog.
# If you are not familiar with recursivity, the next lines might be a bit complicated
def show_tree(collection, char_number=1):
    for subcollection_id, subcollection in collection.children.items():
        print(char_number*"--" + " " + subcollection.id)
        show_tree(subcollection, char_number+1)

print(root.id)
show_tree(root)

default
-- urn:alpheios:arabicLit
---- urn:cts:arabicLit:perseus201003
------ urn:cts:arabicLit:perseus201003.perseus0004
-------- urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1
------ urn:cts:arabicLit:perseus201003.perseus0002
-------- urn:cts:arabicLit:perseus201003.perseus0002.alpheios-text-ara1
------ urn:cts:arabicLit:perseus201003.perseus0003
-------- urn:cts:arabicLit:perseus201003.perseus0003.alpheios-text-ara1
------ urn:cts:arabicLit:perseus201003.perseus0005
-------- urn:cts:arabicLit:perseus201003.perseus0005.alpheios-text-ara1
------ urn:cts:arabicLit:perseus201003.perseus0001
-------- urn:cts:arabicLit:perseus201003.perseus0001.alpheios-text-ara1
---- urn:cts:arabicLit:perseus201001
------ urn:cts:arabicLit:perseus201001.perseus0004
-------- urn:cts:arabicLit:perseus201001.perseus0004.alpheios-text-ara1
------ urn:cts:arabicLit:perseus201001.perseus0001
-------- urn:cts:arabicLit:perseus201001.perseus0001.alpheios-text-ara1
------ urn:cts:arabicLit:perseu

### Printing details about a specific one

In [24]:
# Let's get a random one !
from random import randint
# The index needs to be between 0 and the number of collections
rand_index = randint(0, len(readable_collections))
collection = readable_collections[rand_index]

# Now let's print information ?
label = collection.get_label()

text_id = collection.id
print("Treaing `"+label+"` with id " + text_id)

Treaing `Arabic Reading Lessons` with id urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1


#### What about more detailed informations ? Like the citation scheme ?

In [29]:
def recursive_printing_citation_scheme(citation, char_number=1):
    for subcitation in citation.children:
        print(char_number*"--" + " " + subcitation.name)
        recursive_printing_citation_scheme(subcitation, char_number+1)

print("Maximum citation depth : ", collection.citation.depth)
print("Citation System")
recursive_printing_citation_scheme(collection.citation)

Maximum citation depth :  3
Citation System
-- book
---- section
------ page


### Let's get some references !

In [30]:
reffs = resolver.getReffs(collection.id)
print(reffs)
# Nice !

<DtsReferenceSet (<DtsReference <1> [book]>, <DtsReference <2> [book]>) level:1, citation:<DtsCitation name(book)>>


### Let's get some random passage !


In [33]:
# Let's get a random one !
from random import randint
# The index needs to be between 0 and the number of collections
rand_index = randint(0, len(reffs)-1)
reff = reffs[rand_index]

passage = resolver.getTextualNode(collection.id, reff)
print(passage.id, passage.reference)

# Let's see the XML here
# For that, we need to get the mimetype right :
from MyCapytain.common.constants import Mimetypes
print(passage.export(Mimetypes.XML.TEI))

urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1 <DtsReference <2> [book]>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text xml:lang="ara"><body xml:lang="ara" n="urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1"><div1 type="book" n="2"><div2 n="1" type="section"><div3 n="1" type="page"><l rend="center" xml:lang="en">SECTION I.
    </l><l rend="center" xml:lang="en">Miscellaneous Sentences.</l><l>جُملاَت مُختَلِفَة
</l><l>1 اَلدٌّنيَا دَارُ مَمَرٍ لاَ دَارُ مَقَرٍ * سُلطَان بِلاَ عَدلٍ كَنَهرٍ 
</l><l>بِلاَ مَاءٍ * عَالِم بِلاَ عَمَلٍ كَسَحَابٍ بِلاَ مَطَرٍ * غَنِي بِلاَ سَخَاوةٍ 
</l><l>كِشجِرٍ بِلاَ ثَمَرٍ * إمرأة بِلاَ حَيَاءٍ كَطَعَامٍ بِلاَ مِلحٍ * لاَ تَستَصغِر 
</l><l>عَدُوًّا وَإِن ضَعُفَ * قِلَّةُ الأكلِ يَمنَعُ كَثِيرًا مِن أَعلاَلِ الجِسمِ * 
</l><l>بِالعَمَلِ يُحَصَّلُ الثَّوَابُ لاَ بِالكَسَلِ * 
</l><l>2 مَنْ رَضيَ عَن نَفسِه

### (If you are lucky on random) Get sub passage because it's too big

Let's get the first and the last passage of this text shall we ?

In [34]:
first_passage = passage.first
print(first_passage.export(Mimetypes.XML.TEI))

<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text xml:lang="ara"><body xml:lang="ara" n="urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1"><div1 type="book" n="2"><div2 n="1" type="section"><div3 n="1" type="page"><l rend="center" xml:lang="en">SECTION I.
    </l><l rend="center" xml:lang="en">Miscellaneous Sentences.</l><l>جُملاَت مُختَلِفَة
</l><l>1 اَلدٌّنيَا دَارُ مَمَرٍ لاَ دَارُ مَقَرٍ * سُلطَان بِلاَ عَدلٍ كَنَهرٍ 
</l><l>بِلاَ مَاءٍ * عَالِم بِلاَ عَمَلٍ كَسَحَابٍ بِلاَ مَطَرٍ * غَنِي بِلاَ سَخَاوةٍ 
</l><l>كِشجِرٍ بِلاَ ثَمَرٍ * إمرأة بِلاَ حَيَاءٍ كَطَعَامٍ بِلاَ مِلحٍ * لاَ تَستَصغِر 
</l><l>عَدُوًّا وَإِن ضَعُفَ * قِلَّةُ الأكلِ يَمنَعُ كَثِيرًا مِن أَعلاَلِ الجِسمِ * 
</l><l>بِالعَمَلِ يُحَصَّلُ الثَّوَابُ لاَ بِالكَسَلِ * 
</l><l>2 مَنْ رَضيَ عَن نَفسِهِ كَثُرَ السَّاخِطُ عَلَيهِ * إِذَا كُنتَ 
</l><l>كَذُوبًا فَكُن ذَكُورًا * رَأسُ الُدِين

In [36]:
# Still too big ? Then let's get the child of the child passage
first_passage_last_child = first_passage.last
print(first_passage_last_child.export(Mimetypes.XML.TEI))

<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text xml:lang="ara"><body xml:lang="ara" n="urn:cts:arabicLit:perseus201003.perseus0004.alpheios-text-ara1"><div1 type="book" n="2"><div2 n="1" type="section"><div3 n="9" type="page"><l>جُملاَت مُختَلِفَة
</l><l>لَأَسمَعُ الكَلِمَةَ فَأَحُمٌّ لَهَا ثَلاَثًا مَا يَمْنَعُنِي مِنَ الجَوَابِ عَنهَا 
</l><l>إلاَّ خَوف مِن أن أَسمَعُ شَرًّا مِنهَا * قِيلَ لِبَعضِ الحُكَمَاءِ 
</l><l>مَتَي يُحمَدُ الكِذبُ ؟ قَالَ إذَا جَمَعَ بَيْنَ مُتَقَاطِعِينَ – قِيلَ 
</l><l>فَمَتَي يُذَمٌّ الصِّدقُ؟ قَالَ إذَا كَانَ غِيبَةً – فَمَتَي يَكُونُ 
</l><l>الصَّمتُ خَيرًا مِنَ النٌّطقِ؟ قَالَ عِندَ المِرَاءِ * 
</l><l>Our Lord's Prayer
</l><l>أَبَانَا الذَِّي فِي السَّمَوَاتِ * لِيَتَقَدَّسِ اسمُكَ * لِيَأتِ مَلَكُوتُكَ 
</l><l>لِتَكُن مَشِيَّتُكَ كَمَا فِي السَّمَاءِ كذلِكَ عَلَي الأَرضِ * أَعطِنَا 
</l><l>خُبزَنَا كِفَاةَ يَومِنَا * واغفِر لَنَا ذ